In [1]:
import glob
import pandas as pd
import pandasql as ps
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
data_path = '../data/projections/'
final_avg_file = 'final_averages_jaylee.xlsx'
hash_bball_file = 'hashtag_bball_projections.xlsx'

In [27]:
# SCRAPING DRAFT RESULTS
id = 2458971
league_id = id

In [2]:
# METHODS

def calculate_zscores(df):
    df_final = df.copy()
    punt_cats = ['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']
    
    for col in punt_cats:
        z_score = (df_final[col]-df_final[col].mean()) / df_final[col].std()
        if col == 'tog':
            df_final[col+'_Z'] = -1 * z_score.round(2)
        elif col!='tog':
            df_final[col + '_Z'] = z_score.round(2)
        else:
            pass
        
    df_final = _calculate_total_value(df_final)

    return df_final


def punt_cats(df,punt_list=[]):
    df_final = df.copy()
    
    if not punt_list:
        pass
    else:        
        for cat in punt_list:
            df_final.drop(columns=[cat,cat+'_Z'],inplace=True)
            if cat == 'fg%':
                df_final.drop(columns=['fg','fga'],inplace=True)
            elif cat == 'ft%':
                df_final.drop(columns=['ft','fta'],inplace=True)
            else:
                continue

    df_final = _calculate_total_value(df_final)

    return df_final


def _calculate_total_value(df):
    value_cols = []

    for col in df.columns:
        if col.endswith('_Z'):
            value_cols.append(col)

    df['Total'] = df[value_cols].sum(axis=1)
    df = df.sort_values('Total', ascending=False).reset_index(drop=True)

    return df


def _scrape_results(league_id=0000):
    url = "https://basketball.fantasysports.yahoo.com/nba/{0}/draftresults".format(league_id)
    players = []

    html = urlopen(url)
    soup = BeautifulSoup(html)

    for line in soup.find_all('td'):
        if line.a == None:
            continue
        else:
            players.append(line.a.get_text())

    return players


def find_best_available(df,league_id=0000):
    """
    Scrape draft results and store the player names being taken and recorded

    Take out row based on player name

    Every time you run the returning DataFrame provides only names of those available

    :param df:
    :return:
    """

    players_drafted = _scrape_results(league_id)

    filter = df['Player'].isin(players_drafted)
    df = df.loc[~filter].head(30).reset_index(drop=True)

    return df


def _clean_player_names(df):
    player_names = []
    camel_case_last_names = ('Gilgeous-Alexander','LaVine','VanVleet','DeRozan','DiVincenzo',
                             'McGee',"O'Neale",'LeVert','Carter-Williams','McCollum')
    for player in df.Player:
        
        player_list = player.split(' ')[:3]
        
        if len(player_list) > 2 and (player_list[2].startswith('Jr.') or player_list[2].startswith('II')):
            
            prefix = player_list[2]
            
            if prefix.startswith('Jr.'):
                split_pre =  re.findall('[A-Z][^A-Z]*', prefix)
                split_pre = re.match('[a-zA-Z_.-]+\w*',split_pre[0])
            elif prefix.startswith('II'):
                split_pre = re.findall('I+', prefix)
                split_pre = re.match('[A-Z]\w*',split_pre[0])
            
            player_list[2] = split_pre[0]
            player_string = ' '.join(player_list)
            player_names.append(player_string)
            
        else:
            
            player_list = player_list[:2]
            last_name = player_list[1]
            
            if last_name.startswith(camel_case_last_names) and last_name not in camel_case_last_names:
                split_last = re.match('([^A-Z]*[A-Z]){2}[a-z]*', last_name)
                split_last = re.match('[a-zA-Z_.-]+\w*',split_last[0])
            elif last_name in camel_case_last_names:
                split_last = [last_name]
                split_last = re.match('[A-Z]\w*',split_last[0])
            else:
                split_last = re.findall('[A-Z][^A-Z]*', last_name)
                split_last = re.match('[A-Z]\w*',split_last[0])
            
            player_list[1] = split_last[0]
            player_string = ' '.join(player_list)
            player_names.append(player_string)
   
    player_col = pd.Series(player_names)
    return player_col


def _separate_percentages(col):
    
    list_percentages = []
    list_made = []
    list_attempts = []
    
    for percentage in col:
        stat = percentage.split('(')
        perc = stat[0]
        m = stat[1].split('/')[0]
        a = stat[1].split('/')[1].split(')')[0]
        list_percentages.append(perc)
        list_made.append(m)
        list_attempts.append(a)
    
    col_percent = pd.Series(list_percentages)
    col_made = pd.Series(list_made)
    col_attempt = pd.Series(list_attempts)
    
    return col_percent, col_made, col_attempt
    

def clean_hashtag_df(df):
    df_final = df.copy()
    df_final.drop(columns=['Pos','Tm','G','Mp','Total'],inplace=True)
    df_final['Player'] = _clean_player_names(df_final)
    df_final['fg%'],df_final['fg'],df_final['fga'] = _separate_percentages(df_final['fg%'])    
    df_final['ft%'],df_final['ft'],df_final['fta'] = _separate_percentages(df_final['ft%'])

    # Re-arrange order of columns
    columns = ['Player','fg%','fg','fga','ft%','ft','fta','3pm','ppg',
               'rpg','apg', 'spg','bpg','tog']
    
    for col in df_final.columns[1:]:
        df_final[col] = pd.to_numeric(df_final[col])

    df_final = df_final[columns]
    
    if len(df_final) < len(df):
        print("Warning! Cleaning data removed player(s)")
    
    return df_final


def _scrape_by_team(name,id):
    url = "https://basketball.fantasysports.yahoo.com/nba/{0}/draftresults".format(league_id)
    players_selected = []

    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    for line in soup.find_all('tr'):
        if line.a==None:
            continue
        else:
            if line.find_all('td')[2].get_text() == name:
                players_selected.append(line.a.get_text())
            else:
                continue
                
    return players_selected


def track_team_stats(df,name,id):
    
    players_selected = _scrape_by_team(name,id)
    
    team_df = df.loc[df['Player'].isin(players_selected)]
    team_df = team_df.append(team_df.mean(numeric_only=True), ignore_index=True)
    team_df = team_df.replace(np.nan, 'AVERAGES')
    
    if 'fg' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','fg%']=(team_df['fg']/team_df['fga']).round(2)
    elif 'ft' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','ft%']=(team_df['ft']/team_df['fta']).round(2)
    
    return team_df.round(2)

def track_team_stats_by_list(df,list_players):
    team_df = df.loc[df['Player'].isin(list_players)]
    team_df = team_df.append(team_df.mean(numeric_only=True), ignore_index=True)
    team_df = team_df.replace(np.nan, 'AVERAGES')
    
    if 'fg' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','fg%']=(team_df['fg']/team_df['fga']).round(2)
    elif 'ft' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','ft%']=(team_df['ft']/team_df['fta']).round(2)
    
    return team_df.round(2)

def find_best_cats_available(df, lid, cats=['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']):
    df_final = df.copy()
    # call scrape method
    players_drafted = _scrape_results(lid)
    filter = df_final['Player'].isin(players_drafted)
    df_final = df_final.loc[~filter].head(30).reset_index(drop=True)
    
    # call punt cats method
    all_cats = ['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']
    c = set(cats)
    
    punt_cats_list = [x for x in all_cats if x not in c]
    df_final = punt_cats(df_final,punt_cats_list)
    
    return df_final

In [18]:
hash_bball = pd.read_excel(data_path + hash_bball_file, engine='openpyxl')

In [12]:
punt_kings = pd.read_excel(data_path + final_avg_file, engine = 'openpyxl', index_col =0)

In [13]:
df = punt_kings.copy()

In [17]:
df.iloc[6:]

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,tog
0,Steven Adams,0.56,4.52,8.10,0.60,1.96,3.26,0.09,11.08,8.93,2.40,0.81,1.08,1.59
1,Bam Adebayo,0.53,6.25,11.78,0.70,3.60,5.16,0.21,16.31,9.60,4.73,1.14,1.21,2.63
2,LaMarcus Aldridge,0.49,6.01,12.35,0.81,2.46,3.05,0.95,15.43,6.60,2.05,0.58,1.37,1.34
3,Kyle Alexander,0.47,3.12,6.61,0.74,1.49,2.03,0.36,8.11,4.36,1.51,0.56,0.60,1.17
4,Nickeil Alexander-Walker,0.42,3.21,7.64,0.75,1.23,1.63,1.10,8.75,2.80,2.24,0.64,0.28,1.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,Cody Zeller,0.51,4.25,8.32,0.68,2.02,2.99,0.41,10.93,7.02,1.69,0.68,0.54,1.42
674,Tyler Zeller,0.49,2.03,4.11,0.69,0.84,1.21,0.15,4.91,3.51,0.81,0.35,0.50,0.80
675,Ante Zizic,0.49,2.96,6.01,0.73,1.33,1.81,0.27,7.53,4.31,1.22,0.51,0.54,1.00
676,Ivica Zubac,0.56,3.86,6.85,0.72,1.96,2.70,0.17,9.88,7.98,1.48,0.37,0.99,1.20


In [19]:
# Copy data
df = hash_bball.copy()

In [20]:
# Clean data
df = clean_hashtag_df(df)

In [21]:
# Adjust OVERALL Top Rankings 
df_100 = calculate_zscores(df.head(100))
df_150 = calculate_zscores(df.head(150))
df_200 = calculate_zscores(df.head(200))
df_250 = calculate_zscores(df.head(250))
df_300 = calculate_zscores(df.head(300))

In [22]:
df_200

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,Anthony Davis,0.503,9.0,17.9,0.847,7.3,8.6,1.3,26.6,9.4,...,0.43,0.87,-0.25,2.07,1.56,0.00,1.44,3.33,-0.68,8.77
1,James Harden,0.442,9.5,21.6,0.865,10.4,12.1,4.5,34.0,6.6,...,-0.57,1.10,3.00,3.38,0.41,2.34,1.99,0.30,-3.19,8.76
2,Karl-Anthony Towns,0.501,8.4,16.8,0.804,5.0,6.2,3.3,25.1,11.1,...,0.40,0.33,1.78,1.80,2.26,0.25,0.35,1.71,-1.75,7.13
3,Kawhi Leonard,0.475,9.3,19.7,0.868,6.1,7.1,2.2,27.0,6.9,...,-0.03,1.14,0.66,2.14,0.53,0.78,2.26,-0.10,-0.80,6.58
4,Damian Lillard,0.456,9.0,19.6,0.897,6.8,7.6,4.0,28.7,4.2,...,-0.34,1.51,2.49,2.44,-0.58,2.20,0.35,-0.70,-1.28,6.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Obi Toppin,0.481,5.0,10.4,0.700,1.8,2.5,0.8,12.6,5.3,...,0.07,-1.00,-0.76,-0.42,-0.13,-1.16,-1.02,0.10,0.51,-3.81
196,Jeff Teague,0.443,3.0,6.7,0.876,2.6,3.0,0.7,9.3,1.8,...,-0.56,1.24,-0.87,-1.01,-1.57,0.54,-1.30,-0.70,0.40,-3.83
197,Tristan Thompson,0.520,4.2,8.0,0.605,1.4,2.3,0.2,9.9,8.3,...,0.71,-2.21,-1.37,-0.90,1.10,-0.73,-1.30,0.10,0.51,-4.09
198,Onyeka Okongwu,0.541,4.1,7.5,0.727,1.2,1.6,0.0,9.3,4.3,...,1.05,-0.66,-1.58,-1.01,-0.54,-1.36,-1.30,0.91,0.16,-4.33


In [23]:
punt_to_fg = punt_cats(df_250,['fg%','tog'])

In [24]:
punt_to_ft = punt_cats(df_250,['ft%','tog'])

In [26]:
punt_to_ft

,Player,fg%,fg,fga,3pm,ppg,rpg,apg,spg,bpg,fg%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,James Harden,0.442,9.5,21.6,4.5,34.0,6.6,8.0,1.7,0.8,-0.54,3.24,3.55,0.58,2.60,2.14,0.39,11.96
1,Giannis Antetokounmpo,0.546,10.8,19.8,1.6,29.8,13.8,5.7,1.0,1.0,1.16,0.16,2.82,3.58,1.43,0.25,0.82,10.22
2,Karl-Anthony Towns,0.501,8.4,16.8,3.3,25.1,11.1,3.7,1.1,1.5,0.43,1.96,1.99,2.46,0.40,0.52,1.87,9.63
3,Anthony Davis,0.503,9.0,17.9,1.3,26.6,9.4,3.2,1.5,2.3,0.46,-0.16,2.26,1.75,0.15,1.60,3.56,9.62
4,Luka Doncic,0.465,9.6,20.7,2.9,29.4,9.5,9.1,1.1,0.3,-0.16,1.54,2.75,1.79,3.17,0.52,-0.66,8.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Marcus Morris,0.423,3.1,7.3,1.3,9.8,3.4,0.9,0.5,0.5,-0.85,-0.16,-0.69,-0.75,-1.03,-1.10,-0.24,-4.82
246,Nicolo Melli,0.426,2.4,5.7,1.4,7.2,3.2,1.5,0.6,0.4,-0.80,-0.06,-1.14,-0.84,-0.72,-0.83,-0.45,-4.84
247,Grayson Allen,0.454,3.4,7.6,1.9,9.9,2.4,1.6,0.3,0.1,-0.34,0.48,-0.67,-1.17,-0.67,-1.64,-1.08,-5.09
248,Dewayne Dedmon,0.426,2.6,6.1,0.6,6.3,4.9,0.5,0.4,0.9,-0.80,-0.91,-1.30,-0.13,-1.24,-1.37,0.61,-5.14


In [ ]:
# WORK FLOW

# 1) calculate z-score values with all categories and that is the main data frame
# 2) create a df for players adjusted values based on punting categories

In [11]:
punt_to_fg = punt_cats(df_250,['fg%','tog'])
punt_to_fg_blk = punt_cats(df_300,['fg%','tog','bpg'])
punt_to_fg_reb = punt_cats(df_300,['fg%','tog','rpg'])
punt_to_fg_ast_stl_3pm = punt_cats(df_300,['fg%','tog','apg','spg','3pm'])
punt_none = punt_cats(df_300)

In [28]:
find_best_available(df_250,league_id)

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,Anthony Davis,0.503,9.0,17.9,0.847,7.3,8.6,1.3,26.6,9.4,...,0.46,0.89,-0.16,2.26,1.75,0.15,1.60,3.56,-0.88,9.63
1,James Harden,0.442,9.5,21.6,0.865,10.4,12.1,4.5,34.0,6.6,...,-0.54,1.11,3.24,3.55,0.58,2.60,2.14,0.39,-3.45,9.62
2,Karl-Anthony Towns,0.501,8.4,16.8,0.804,5.0,6.2,3.3,25.1,11.1,...,0.43,0.36,1.96,1.99,2.46,0.40,0.52,1.87,-1.98,8.01
3,Kawhi Leonard,0.475,9.3,19.7,0.868,6.1,7.1,2.2,27.0,6.9,...,0.00,1.14,0.79,2.33,0.71,0.97,2.41,-0.03,-1.01,7.31
4,Damian Lillard,0.456,9.0,19.6,0.897,6.8,7.6,4.0,28.7,4.2,...,-0.31,1.50,2.71,2.62,-0.42,2.45,0.52,-0.66,-1.49,6.92
5,Jayson Tatum,0.455,9.0,19.7,0.813,3.9,4.8,3.1,24.9,7.1,...,-0.32,0.47,1.75,1.96,0.79,0.66,1.33,0.61,-0.76,6.49
6,Giannis Antetokounmpo,0.546,10.8,19.8,0.664,6.6,10.0,1.6,29.8,13.8,...,1.16,-1.35,0.16,2.82,3.58,1.43,0.25,0.82,-2.47,6.40
7,Stephen Curry,0.459,8.0,17.5,0.911,6.2,6.8,4.1,26.2,5.4,...,-0.26,1.67,2.82,2.19,0.08,1.79,0.79,-0.24,-2.47,6.37
8,Nikola Jokic,0.519,7.9,15.2,0.819,3.5,4.3,1.2,20.6,10.3,...,0.72,0.54,-0.27,1.20,2.12,2.25,1.06,0.18,-1.74,6.06
9,Paul George,0.432,7.9,18.3,0.875,4.5,5.1,3.7,24.0,6.4,...,-0.70,1.23,2.39,1.80,0.50,0.51,2.14,-0.45,-1.37,6.05


In [42]:
find_best_available(punt_to_fgsign,league_id)

,Player,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,Devonte' Graham,0.825,2.5,3.0,3.0,15.5,2.9,5.8,1.1,0.2,0.62,1.65,0.31,-0.96,1.48,0.52,-0.87,2.75
1,Elfrid Payton,0.686,1.6,2.3,0.6,11.1,5.1,7.7,1.8,0.5,-1.08,-0.91,-0.46,-0.04,2.45,2.41,-0.24,2.13
2,Darius Bazley,0.761,1.4,1.9,2.1,13.8,6.8,1.6,0.9,1.2,-0.17,0.69,0.01,0.66,-0.67,-0.02,1.24,1.74
3,Duncan Robinson,0.902,0.9,1.0,3.8,13.7,3.2,1.7,0.7,0.3,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.38
4,Terry Rozier,0.872,2.3,2.6,2.5,15.5,3.5,3.0,1.0,0.2,1.19,1.11,0.31,-0.71,0.04,0.25,-0.87,1.32
5,Josh Richardson,0.807,2.3,2.8,2.1,14.9,3.3,2.7,1.1,0.8,0.40,0.69,0.21,-0.79,-0.11,0.52,0.39,1.31
6,Bojan Bogdanovic,0.899,4.0,4.4,3.0,19.9,4.1,2.1,0.5,0.1,1.52,1.65,1.08,-0.46,-0.42,-1.10,-1.08,1.19
7,Norman Powell,0.848,2.5,2.9,2.2,15.8,3.5,1.8,1.2,0.4,0.90,0.79,0.36,-0.71,-0.57,0.79,-0.45,1.11
8,Evan Fournier,0.817,2.7,3.3,2.6,17.4,2.6,3.2,1.1,0.2,0.52,1.22,0.64,-1.09,0.15,0.52,-0.87,1.09
9,LaMelo Ball,0.727,1.8,2.4,1.6,13.3,5.1,6.1,1.2,0.2,-0.58,0.16,-0.07,-0.04,1.63,0.79,-0.87,1.02


In [45]:
track_team_stats(punt_to_fg,"Josh",league_id)

,Player,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,James Harden,0.86,10.40,12.10,4.5,34.00,6.60,8.00,1.70,0.80,1.11,3.24,3.55,0.58,2.60,2.14,0.39,13.61
1,Kyrie Irving,0.91,4.20,4.60,2.5,24.60,4.20,6.10,1.30,0.50,1.71,1.11,1.91,-0.42,1.63,1.06,-0.24,6.76
2,Kristaps Porzingis,0.80,4.00,5.00,2.6,20.70,9.40,1.80,0.70,2.00,0.26,1.22,1.22,1.75,-0.57,-0.56,2.93,6.25
3,Jusuf Nurkic,0.79,3.40,4.30,0.4,15.80,9.80,3.60,1.20,1.60,0.14,-1.12,0.36,1.92,0.35,0.79,2.08,4.52
4,Robert Covington,0.82,1.20,1.50,2.5,12.30,7.00,1.40,1.60,1.20,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,4.45
5,Brook Lopez,0.86,2.00,2.30,1.8,12.70,4.60,1.60,0.70,2.40,1.11,0.37,-0.18,-0.25,-0.67,-0.56,3.77,3.59
6,Marcus Smart,0.84,2.10,2.50,2.3,13.00,3.80,4.60,1.60,0.50,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,3.43
7,Devonte' Graham,0.82,2.50,3.00,3.0,15.50,2.90,5.80,1.10,0.20,0.62,1.65,0.31,-0.96,1.48,0.52,-0.87,2.75
8,Elfrid Payton,0.69,1.60,2.30,0.6,11.10,5.10,7.70,1.80,0.50,-1.08,-0.91,-0.46,-0.04,2.45,2.41,-0.24,2.13
9,Darius Bazley,0.76,1.40,1.90,2.1,13.80,6.80,1.60,0.90,1.20,-0.17,0.69,0.01,0.66,-0.67,-0.02,1.24,1.74


In [44]:
Team_A = ['James Harden','Kyrie Irving','Jusuf Nurkic','Kristaps Porzingis','Robert Covington',
          'Marcus Smart','Brook Lopez',"Devonte' Graham",'Bojan Bogdanovic','Josh Richardson','Elfrid Payton',
         'Terry Rozier','Darius Bazley']

Team_B = ['James Harden','Kyrie Irving','Ben Simmons','LaMarcus Aldridge','Jonas Valanciunas',
          'Derrick White','Lauri Markkanen','Davis Bertans','Tyler Herro','DeAndre Jordan',
          'Larry Nance Jr.','Terry Rozier','Marquese Chriss']

In [58]:
def track_team_stats_by_list_tot(df,list_players):
    team_df = df.loc[df['Player'].isin(list_players)]
    team_df = team_df.append(team_df.sum(numeric_only=True), ignore_index=True)
    team_df = team_df.replace(np.nan, 'TOTAL')
    
#     if 'fg' in team_df.columns:
#         team_df.loc[team_df['Player']=='TOTAL','fg%']=(team_df['fg']/team_df['fga']).round(2)
#     elif 'ft' in team_df.columns:
#         team_df.loc[team_df['Player']=='TOTAL','ft%']=(team_df['ft']/team_df['fta']).round(2)
    
    return team_df.round(2)

def replace_perc(df):
    df.loc[df['Player']=='TOTAL','fg%']=(df['fg']/df['fga']).round(2)
    df.loc[df['Player']=='TOTAL','ft%']=(df['ft']/df['fta']).round(2)
    return df

In [64]:
replace_perc(track_team_stats_by_list_tot(df_250,Team_A))

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,James Harden,0.44,9.5,21.6,0.86,10.4,12.1,4.5,34.0,6.6,...,-0.54,1.11,3.24,3.55,0.58,2.60,2.14,0.39,-3.45,9.62
1,Kristaps Porzingis,0.44,7.0,16.1,0.80,4.0,5.0,2.6,20.7,9.4,...,-0.62,0.26,1.22,1.22,1.75,-0.57,-0.56,2.93,0.21,5.84
2,Kyrie Irving,0.48,9.0,18.8,0.91,4.2,4.6,2.5,24.6,4.2,...,0.07,1.71,1.11,1.91,-0.42,1.63,1.06,-0.24,-1.01,5.82
3,Jusuf Nurkic,0.50,6.0,12.1,0.79,3.4,4.3,0.4,15.8,9.8,...,0.38,0.14,-1.12,0.36,1.92,0.35,0.79,2.08,-0.40,4.50
4,Brook Lopez,0.43,4.5,10.3,0.86,2.0,2.3,1.8,12.7,4.6,...,-0.67,1.11,0.37,-0.18,-0.25,-0.67,-0.56,3.77,0.95,3.87
5,Robert Covington,0.42,4.3,10.2,0.82,1.2,1.5,2.5,12.3,7.0,...,-0.83,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,-0.03,3.59
6,Marcus Smart,0.38,4.3,11.4,0.84,2.1,2.5,2.3,13.0,3.8,...,-1.57,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,-0.15,1.71
7,Darius Bazley,0.44,5.1,11.8,0.76,1.4,1.9,2.1,13.8,6.8,...,-0.62,-0.17,0.69,0.01,0.66,-0.67,-0.02,1.24,0.21,1.33
8,Elfrid Payton,0.44,4.5,10.4,0.69,1.6,2.3,0.6,11.1,5.1,...,-0.64,-1.08,-0.91,-0.46,-0.04,2.45,2.41,-0.24,-0.88,0.61
9,Josh Richardson,0.44,5.3,12.1,0.81,2.3,2.8,2.1,14.9,3.3,...,-0.59,0.40,0.69,0.21,-0.79,-0.11,0.52,0.39,-0.52,0.20


In [63]:
replace_perc(track_team_stats_by_list_tot(df_250,Team_B))

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,James Harden,0.44,9.5,21.6,0.86,10.4,12.1,4.5,34.0,6.6,...,-0.54,1.11,3.24,3.55,0.58,2.60,2.14,0.39,-3.45,9.62
1,Kyrie Irving,0.48,9.0,18.8,0.91,4.2,4.6,2.5,24.6,4.2,...,0.07,1.71,1.11,1.91,-0.42,1.63,1.06,-0.24,-1.01,5.82
2,Ben Simmons,0.56,6.8,12.0,0.63,3.3,5.2,0.0,16.9,8.0,...,1.46,-1.72,-1.55,0.56,1.17,2.66,3.22,-0.03,-2.10,3.67
3,LaMarcus Aldridge,0.48,6.3,13.0,0.84,2.8,3.4,1.3,16.6,7.0,...,0.10,0.74,-0.16,0.50,0.75,-0.31,-0.56,1.87,0.34,3.27
4,Derrick White,0.47,4.7,10.0,0.84,3.2,3.8,1.5,14.1,4.0,...,-0.13,0.78,0.05,0.07,-0.50,0.71,0.25,1.03,0.09,2.35
5,Lauri Markkanen,0.44,6.0,13.8,0.84,3.3,3.9,2.4,17.7,7.2,...,-0.65,0.74,1.01,0.70,0.83,-0.67,-0.02,-0.24,-0.15,1.55
6,Jonas Valanciunas,0.56,6.1,10.7,0.75,2.1,2.8,0.5,14.7,10.4,...,1.46,-0.34,-1.01,0.17,2.17,-0.47,-1.37,0.82,-0.15,1.28
7,DeAndre Jordan,0.67,3.4,5.1,0.68,1.5,2.2,0.0,8.3,9.9,...,3.14,-1.12,-1.55,-0.95,1.96,-0.57,-1.64,0.82,0.46,0.55
8,Davis Bertans,0.43,4.2,9.8,0.82,1.6,2.0,3.2,13.3,3.9,...,-0.67,0.61,1.86,-0.07,-0.54,-0.72,-0.83,-0.24,0.95,0.35
9,Terry Rozier,0.43,5.4,12.6,0.87,2.3,2.6,2.5,15.5,3.5,...,-0.77,1.19,1.11,0.31,-0.71,0.04,0.25,-0.87,-0.40,0.15


In [78]:
z_cats = ['Player','fg%_Z','ft%_Z','3pm_Z','ppg_Z','rpg_Z','apg_Z','spg_Z','bpg_Z','tog_Z']

In [81]:
def calculate_net_value(df):
    df_final = df.copy()
    
    df_final = df_final[:-1]
    df_f
    
    return df

In [84]:
jaylee[:-1]

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,Paul George,0.43,7.9,18.3,0.88,4.5,5.1,3.7,24.0,6.4,...,-0.70,1.23,2.39,1.80,0.50,0.51,2.14,-0.45,-1.37,6.05
1,Fred VanVleet,0.41,6.1,14.8,0.84,3.2,3.8,2.8,18.2,3.8,...,-1.03,0.78,1.43,0.78,-0.59,1.99,2.68,-0.66,-0.64,4.74
2,Trae Young,0.44,7.8,17.6,0.87,7.8,9.0,3.4,26.8,3.5,...,-0.52,1.12,2.07,2.29,-0.71,3.42,0.25,-1.08,-2.59,4.25
3,Robert Covington,0.42,4.3,10.2,0.82,1.2,1.5,2.5,12.3,7.0,...,-0.83,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,-0.03,3.59
4,D'Angelo Russell,0.43,7.1,16.6,0.82,3.2,3.9,3.4,20.9,3.6,...,-0.75,0.61,2.07,1.26,-0.67,1.63,0.79,-0.66,-1.37,2.91
5,Duncan Robinson,0.47,4.5,9.6,0.90,0.9,1.0,3.8,13.7,3.2,...,-0.11,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.07,2.34
6,Richaun Holmes,0.63,4.6,7.2,0.78,1.9,2.4,0.0,11.0,7.2,...,2.51,0.02,-1.55,-0.48,0.83,-1.03,-0.29,1.24,0.82,2.07
7,Marcus Smart,0.38,4.3,11.4,0.84,2.1,2.5,2.3,13.0,3.8,...,-1.57,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,-0.15,1.71
8,Wendell Carter Jr.,0.51,4.6,9.0,0.74,2.9,3.9,0.2,12.3,9.9,...,0.54,-0.40,-1.33,-0.25,1.96,-0.83,-0.02,1.24,-0.03,0.88
9,Maxi Kleber,0.49,3.7,7.4,0.83,1.2,1.5,1.8,10.3,5.6,...,0.26,0.73,0.37,-0.60,0.16,-0.77,-1.64,1.24,1.07,0.82
